# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
# import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline


from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [3]:
previsao_de_renda = 'https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2013%20Regress%C3%A3o%20II/database/previsao_de_renda.csv'

df = pd.read_csv(previsao_de_renda, index_col=0)
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usage: 1

### Limpeza dos dados

In [5]:
df = df.dropna()
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,2016-03-01,11477,M,True,False,0,Empresário,Superior incompleto,Casado,Casa,32,9.849315,2.0,1592.57
14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [6]:
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)
df

,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12085,2016-03-01,11477,M,True,False,0,Empresário,Superior incompleto,Casado,Casa,32,9.849315,2.0,1592.57
12086,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
12087,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
12088,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [7]:
df.columns

Index(['data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'renda'],
      dtype='object')

In [8]:
df = df[['sexo', 'posse_de_veiculo', 'posse_de_imovel',
       'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil',
       'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia',
       'renda']]
df

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...
12085,M,True,False,0,Empresário,Superior incompleto,Casado,Casa,32,9.849315,2.0,1592.57
12086,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
12087,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
12088,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

#### 1) Seprando a base em treinamento e teste

In [152]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

modelo ='''renda ~ sexo + posse_de_veiculo 
        + posse_de_imovel 
        + qtd_filhos 
        + tipo_renda 
        + educacao 
        + estado_civil 
        + tipo_residencia 
        + idade 
        + tempo_emprego 
        + qt_pessoas_residencia'''

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=df_test)

#### 2) Rodando uma regularização Ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1].

In [153]:
#Regularização ridge
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)
reg.summary()

ValueError: shapes (3023,24) and (25,) not aligned: 24 (dim 1) != 25 (dim 0)

###### .

#### 3) Regressão LASSO com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1]. 


In [154]:
# Regularização Lasso
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                                 refit=True, 
                                 L1_wt=1,  # lasso fit
                                 alpha=alpha)
       
    r2 = reg.rsquared
    r2_y_pred.append(r2)    
    
    print(f'Alpha {alpha}: \nR-squared = {round(r2*100,3)}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))
reg.summary()

Alpha 0: 
R-squared = 26.471

Alpha 0.001: 
R-squared = 26.471

Alpha 0.005: 
R-squared = 26.471

Alpha 0.01: 
R-squared = 26.47

Alpha 0.05: 
R-squared = 26.471

Alpha 0.1: 
R-squared = 26.47

   alpha        𝑅2
0  0.000  0.264706
1  0.001  0.264706
2  0.005  0.264706
4  0.050  0.264706
3  0.010  0.264704
5  0.100  0.264704


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.265
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     135.6
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        11:51:32   Log-Likelihood:                -94120.
No. Observations:                9067   AIC:                         1.883e+05
Df Residuals:                    9043   BIC:                         1.885e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -6081.5627   2740.690     -2.219      0.027   -1.15e+04    -709.189
sexo[T.M]                         6140.5738    176.727     34.746      0.000    5794.150    6486.998
posse_de_veiculo[T.True]                  0          0        nan        nan           0           0
posse_de_imovel[T.True]            436.9654    177.400      2.463      0.014      89.220     784.710
tipo_renda[T.Bolsista]           -1934.1599   3194.337     -0.605      0.545   -8195.784    4327.464
tipo_renda[T.Empresário]           733.5128    189.338      3.874      0.000     362.367    1104.659
tipo_renda[T.Pensionista]        -2504.6666   2612.707     -0.959      0.338   -7626.164    2616.831
tipo_renda[T.Servidor público]    -202.6241    285.695     -0.709      0.478    -762.652     357.403
educacao[T.Pós graduação]          986.1124   1966.456      0.501      0.616   -2868.586    4840.811
educacao[T.Secundário]             309.2405    901.649      0.343      0.732   -1458.196    2076.677
educacao[T.Superior completo]      914.6415    905.252      1.010      0.312    -859.857    2689.140
educacao[T.Superior incompleto]    114.7162    976.130      0.118      0.906   -1798.720    2028.153
estado_civil[T.Separado]          1105.5528   1284.750      0.861      0.390   -1412.848    3623.953
estado_civil[T.Solteiro]           796.6536   1252.417      0.636      0.525   -1658.367    3251.674
estado_civil[T.União]             -447.7711    315.843     -1.418      0.156   -1066.894     171.352
estado_civil[T.Viúvo]              808.3797   1344.362      0.601      0.548   -1826.874    3443.634
tipo_residencia[T.Casa]           -373.2549    675.308     -0.553      0.580   -1697.011     950.501
tipo_residencia[T.Com os pais]    -161.9153    753.794     -0.215      0.830   -1639.522    1315.691
tipo_residencia[T.Comunitário]    -792.2112   1337.906     -0.592      0.554   -3414.810    1830.387
tipo_residencia[T.Estúdio]         207.6517   1284.241      0.162      0.872   -2309.752    2725.055
tipo_residencia[T.Governamental]   330.3126    823.343      0.401      0.688   -1283.626    1944.251
qtd_filhos                       -1363.0199   1239.597     -1.100      0.272   -3792.910    1066.870
idade                               49.9320     10.149      4.920      0.000      30.038      69.826
tempo_emprego                      580.6316     13.135     44.203      0.000     554.883     606.380
qt_pessoas_residencia             1445.6374   1236.840      1.169      0.243    -978.850    3870.125
==============================================================================
Omnibus:                    13307.497   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8922938.031
Skew:             

###### Um insight importante da análise dos modelos de regressão com regularização LASSO testados é que não houve mudanças consideráveis nos resultados. Os valores de R² ficaram muito próximos de 26,47, indicando uma consistência no desempenho dos modelos, com uma leve exceção nos alphas 0.01 e 0.1 que deram um R² 0.001 a menos que os outros.

##### 4) Rode um modelo stepwise e avaliando o  𝑅²  na base de testes. 

In [155]:
df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [156]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('\n\nresulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 2.97268e-83
#############
['tempo_emprego']
Add  sexo_M                         with p-value 5.76016e-87
#############
['tempo_emprego', 'sexo_M']
Add  tipo_renda_Empresário          with p-value 0.00310946
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário']
Add  tipo_renda_Servidor público    with p-value 0.00599596
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'tipo_renda_Servidor público']
Add  educacao_Superior completo     with p-value 0.0266685
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'tipo_renda_Servidor público', 'educacao_Superior completo']
Add  educacao_Secundário            with p-value 0.0473896
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'tipo_renda_Servidor público', 'educacao_Superior completo', 'educacao_Secundário']
#############
['tempo_emprego', 'sexo_M', 'tipo_renda_Empresário', 'tipo_renda_Servidor público', 'educacao_Superior compl

In [149]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)
print(f'R-squared: {r2_score(y_test, y_pred)}')

R-squared: 0.21921784286829793


In [150]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))
reg.summary()

Alpha 0: 
R-squared = 0.21921784286829804

Alpha 0.001: 
R-squared = 0.2192729211769654

Alpha 0.005: 
R-squared = 0.21943535149237647

Alpha 0.01: 
R-squared = 0.21952151833033162

Alpha 0.05: 
R-squared = 0.2172544862835254

Alpha 0.1: 
R-squared = 0.21113531766517402

   alpha        𝑅2
3  0.010  0.219522
2  0.005  0.219435
1  0.001  0.219273
0  0.000  0.219218
4  0.050  0.217254
5  0.100  0.211135


NotImplementedError: 

In [151]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
print(pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False))
reg.summary()

Alpha 0: 
R-squared = 0.21921784286829793

Alpha 0.001: 
R-squared = 0.21921784286829793

Alpha 0.005: 
R-squared = 0.21921784286829793

Alpha 0.01: 
R-squared = 0.21921784286829793

Alpha 0.05: 
R-squared = 0.21921784286829793

Alpha 0.1: 
R-squared = 0.21921784286829793

   alpha        𝑅2
0  0.000  0.219218
1  0.001  0.219218
2  0.005  0.219218
3  0.010  0.219218
4  0.050  0.219218
5  0.100  0.219218


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  renda   R-squared (uncentered):                -345.828
Model:                            OLS   Adj. R-squared (uncentered):           -346.057
Method:                 Least Squares   F-statistic:                             -1506.
Date:                Wed, 29 May 2024   Prob (F-statistic):                        1.00
Time:                        11:50:45   Log-Likelihood:                         -94158.
No. Observations:                9067   AIC:                                  1.883e+05
Df Residuals:                    9061   BIC:                                  1.884e+05
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
tempo_emprego                 588.3168     12.176     48.319      0.000     564.450     612.184
sexo_M                       5951.3344    168.819     35.253      0.000    5620.411    6282.258
tipo_renda_Empresário         574.2091    185.218      3.100      0.002     211.141     937.278
tipo_renda_Servidor público  -268.6618    285.393     -0.941      0.347    -828.097     290.774
educacao_Superior completo   -117.3163    188.166     -0.623      0.533    -486.164     251.531
educacao_Secundário          -592.0267    166.589     -3.554      0.000    -918.580    -265.474
==============================================================================
Omnibus:                    13359.759   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9021563.140
Skew:                           8.746   Prob(JB):                         0.00
Kurtosis:                     156.537   Cond. No.                         36.3
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""